# **BM455 Yapay Zekaya Giriş**


### FİNAL UYGULAMA ÖDEVİ

In [2]:
import sys
import os
import re
import string
import numpy as np
import pandas as pd
from decimal import Decimal
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
#Veri okuma
def read_data(folder_name):
    spam_mails = []
    ham_mails = []
    #sadece txt olan dosya isimlerini al
    ham_filenames = [name for name in sorted(os.listdir(os.path.join(folder_name,'ham'))) if name.endswith(".txt")]
    spam_filenames = [name for name in sorted(os.listdir(os.path.join(folder_name,'spam'))) if name.endswith(".txt")]
    
    #her ham dosya için oku ve ham_mails içine at
    for filename in ham_filenames:
        with open(os.path.join(folder_name, 'ham', filename), 'r', encoding='latin-1') as f:
            messageBody = f.read()
            ham_mails.append(messageBody)
            
    #her spam dosya için oku ve spam_mails içine at
    for filename in spam_filenames:
        with open(os.path.join(folder_name, 'spam', filename), 'r', encoding='latin-1') as f:
            messageBody = f.read()
            spam_mails.append(messageBody)

    return (spam_mails, ham_mails)


In [4]:
#Spam ve ham verileri liste olarak al
train_spam_mail, train_ham_mail = read_data('data_spam_ham_emails\\training\\')

#ham ve spam dataframe oluştur
ham_df = pd.DataFrame(train_ham_mail, columns = ['mail'])
spam_df = pd.DataFrame(train_spam_mail, columns = ['mail'])


In [5]:
spam_words = []
ham_words = []
#Spam mail içerisindeki kelimeleri temizle ve spam kelimelerini bul
def findSpamWords(spam_df):
    global spam_words
    #kelimeye ayır
    word_tokenized = word_tokenize(spam_df)
    #kelimele lower yap ve temizle
    words = [word.lower() for word in word_tokenized if word.lower().isalpha() and len(word)>3]
    spam_words += words

#Ham mail içerisindeki kelimeleri temizle ve ham kelimelerini bul    
def findHamWords(ham_df):
    global ham_words
    #kelimeye ayır
    word_tokenized = word_tokenize(ham_df)
    #kelimele lower yap ve temizle
    words = [word.lower() for word in  word_tokenized if word.lower().isalpha() and len(word)>3]
    ham_words = ham_words + words

spam_df["mail"].apply(findSpamWords)
ham_df["mail"].apply(findHamWords)

spam_words = np.array(spam_words)
print("Top 10 Spam Kelimeleri:\n")
print(pd.Series(spam_words).value_counts().head(n = 10))

ham_words = np.array(ham_words)
print("\nTop 10 Ham Kelimeleri:\n")
print(pd.Series(ham_words).value_counts().head(n = 10))

Top 10 Spam Kelimeleri:

this       1353
your       1350
that        862
with        735
will        682
from        601
have        594
subject     454
email       389
free        382
dtype: int64

Top 10 Ham Kelimeleri:

enron       834
subject     711
vince       692
that        615
with        613
this        544
will        542
have        482
kaminski    393
your        388
dtype: int64


In [6]:
def cleanMail(mails):
    allword = []
    #Gelen her spam ve ham maillerini temizle
    for message in mails:
        clean_words = clean(message)
        # sadece kelimeyi içeriyorsa listeye ekle
        if not bool(clean_words):
            continue
        allword.append(clean_words)
    return allword

def clean(txt):
    words = dict()
    for word in txt.split():
        lword = word.lower()
        #Subject hepsinde olduğu için çıkar
        #Etkisiz kelimeleri çıkar (stopwords)
        if (lword !='subject' and lword not in stopwords.words("english") and lword.isalpha() and len(lword)>3 and lword not in string.punctuation):
            words[lword] = words.get(lword, 0) + 1
    return words

#Spam ve Ham için her kelimenin olasılığını hesapla
def train(spamword, hamword):    
    spam = dict()
    ham= dict()
    possibilities = dict(dict())
    #Spam kelime kaç tane
    for words in spamword:
        for word in words:
            spam[word] = spam.get(word, 0) + 1
    #Ham kelime kaç tane
    for words in hamword:
        for word in words:
            ham[word] = ham.get(word, 0) + 1
    #tüm kelimeleri farklı (union) olanları birleştir
    all_words = set(ham).union(set(spam))
    
    for word in all_words:
        spam_count = spam.get(word,0)
        ham_count = ham.get(word,0)
        #eğer hiç yoksa 0/xden olasılık hep sıfır olacaktı bu durum başarı oranını etkiliyor
        #pay sıfır olmaması için +1 ekledim sapma azaltmak için paydaya 2 ekledim
        #1/2 --> 2/4 gibi 
        possibilities[word] = {'spam': ((spam_count + 1) / (len(spamword) + 2)),'ham': ((ham_count + 1) / (len(hamword) + 2))}
    return possibilities

In [31]:
#Ham ve Spam verilerini temizle
hamwords=cleanMail(train_ham_mail)
spamwords=cleanMail(train_spam_mail)

#Tüm kelimelerin ham ve spam olasılıklarını bul
possibilities=train(spamwords,hamwords)
possibilities

{'mojego': {'spam': 0.0026595744680851063, 'ham': 0.007957559681697613},
 'profiles': {'spam': 0.010638297872340425, 'ham': 0.005305039787798408},
 'breathing': {'spam': 0.018617021276595744, 'ham': 0.002652519893899204},
 'gland': {'spam': 0.005319148936170213, 'ham': 0.005305039787798408},
 'experience': {'spam': 0.0398936170212766, 'ham': 0.06631299734748011},
 'author': {'spam': 0.010638297872340425, 'ham': 0.007957559681697613},
 'petrochemical': {'spam': 0.007978723404255319, 'ham': 0.007957559681697613},
 'filter': {'spam': 0.005319148936170213, 'ham': 0.002652519893899204},
 'interestedin': {'spam': 0.005319148936170213, 'ham': 0.002652519893899204},
 'concentrations': {'spam': 0.005319148936170213, 'ham': 0.002652519893899204},
 'traveling': {'spam': 0.010638297872340425, 'ham': 0.01856763925729443},
 'surfnet': {'spam': 0.005319148936170213, 'ham': 0.002652519893899204},
 'wzgledy': {'spam': 0.0026595744680851063, 'ham': 0.007957559681697613},
 'herbaceousness': {'spam': 0.00

In [55]:
def spam_possibilities(mail):
    
    #mail kelimelere ayır ve temizle
    words = clean(mail)

    # P(word|spam)
    p_spamword = 1

    # P(word|ham)
    p_hamword = 1
    
    #Gelen test mailindeki tüm kelimelere bak 
    #Eğer kelime train kelimeleri içerisinde varsa olasılığını al 
    #Yoksa 1-değerini al
    #email kelimesi possibilities değeri 0.06 olsun ve possibilities içinde var
    #eğer test verisinde gelen kelimeler içerisinde email yoksa 0.04 al
    #Tüm olasılıkları çarp ve genel spam ve ham olasılığını bul                
    # p = p1 p2 ... pN
    for word, posibility in possibilities.items():
        #Virgülden sonraki basamak sayısını arttırmak için Decimal kütüphanesi kullanılır
        spam_posibility = Decimal(posibility['spam'])
        ham_posibility = Decimal(posibility['ham'])
        p_spamword *= (spam_posibility if word in words else  1 - spam_posibility)
        p_hamword  *= (ham_posibility  if word in words else  1 - ham_posibility)

    # P(spam)
    pSpam = Decimal(len(train_spam_mail) / len(train_spam_mail) + len(train_ham_mail))

    # P(ham)
    pHam = Decimal(len(train_ham_mail) / len(train_spam_mail) + len(train_ham_mail))
     # Probability that a message containing a given word is spam
     #                      P(word|spam)*P(spam)
     # P(spam|word) = -----------------------------------
     #          P(word|spam)*P(spam) + P(word|ham)*P(ham)
    
    result = (p_spamword * pSpam) / (p_spamword * pSpam + p_hamword * pHam)
    return result


def test(test_spam_mail,test_ham_mail):
    
    #TP: Spam mailin tahmini spam
    TP = 0
    #TN: Ham mailin tahmini ham
    TN = 0
    #FN: Spam mail tahmini ham
    FN = 0
    #FP: Ham mail tahmini spam
    FP = 0
    
    #Spam mailler için spam olasılığı hesaplandı
    for mail in test_spam_mail:
        spam_posibility = spam_possibilities(mail)
        #Eğer spam olasılığı 0.5'den yüksekse spamdır ve kendisi de spam (TP) 
        if spam_posibility>0.5:
            TP += 1
        #Eğer spam olasılığı 0.5'den düşükse hamdır ama kendisi spam (FN)
        elif spam_posibility<0.5:
            FN += 1
    #Ham mailler için spam olasılığı hesaplandı
    for mail in test_ham_mail:
        ham_posibility = spam_possibilities(mail)
        #Eğer spam olasılığı 0.5'den düşükse hamdır ve kendisi de ham (TN) 
        if ham_posibility<0.5:
            TN += 1
        #Eğer spam olasılığı 0.5'den yüksekse spamdır ama kendisi ham (FP) 
        elif ham_posibility>0.5:
            FP += 1

    print("TP: %s" % str(TP))
    print("FP: %s" % str(FP))
    print("TN: %s" % str(TN))
    print("FN: %s" % str(FN))
    
    #             TP + TN
    # Accuracy = -----------------      
    #            TP + FP + TN + FN
    accuracy = int(((TP + TN) / (TP + FP + TN + FN)) * 100)
    print('Accuracy: %%%s' % str(accuracy))
    #Precision:Positif olarak tahminlediğimiz değerlerin gerçekten kaç tanesi Positif olduğunu gösterir.
    #             TP 
    # Precision = ----------    
    #            TP + FP 
    precision = int(((TP) / (TP + FP)) * 100)
    print('Precision: %%%s' % str(precision))
    
    #Recall: Positif olarak tahmin etmemiz gereken işlemlerin ne kadarını Positif olarak tahmin ettiğimizi gösterir.
    #             TP 
    # Recall = ----------    
    #            TP + FN
    recall = int(((TP) / (TP + FN)) * 100)
    print('Recall: %%%s' % str(recall))
    #F1 Score: Precision ve Recall değerlerinin harmonik ortalamasını gösterir.
    #                 precision*recall 
    # F1 Score = 2* ----------------------  
    #                precision+recall 
    f1_score = int(((TP) / (TP + FN)) * 100)
    print('F1 Score: %%%s' % str(f1_score))
    
    
def predict(test_mail):
    for each in test_mail:
        spam_posibility = spam_possibilities(each)
        if spam_posibility > 0.5:
            print("Spam")
        else:
            print("Ham")


In [56]:
#Veri 0/1 olacak şekilde okuma
def read_test_data(folder_name):
    spam_mails = []
    ham_mails = []
    ham_filenames=[]
    spam_filenames=[]
    for name in os.listdir(folder_name):
        #sadece txt olan  ve başında 0 olan dosya isimlerini ham listesine ekle
        if name.endswith(".txt") and name.startswith("0"):
            ham_filenames.append(name)
        #sadece txt olan  ve başında 0 olmayan dosya isimlerini spam listesine ekle
        if name.endswith(".txt") and name.startswith("0") == False:
            spam_filenames.append(name)

    #her ham dosya için oku ve ham_mails içine at
    for filename in ham_filenames:
        with open(os.path.join(folder_name, filename), 'r', encoding='latin-1') as f:
            messageBody = f.read()
            ham_mails.append(messageBody)
            
    #her spam dosya için oku ve spam_mails içine at
    for filename in spam_filenames:
        with open(os.path.join(folder_name, filename), 'r', encoding='latin-1') as f:
            messageBody = f.read()
            spam_mails.append(messageBody)

    return (spam_mails, ham_mails)

In [57]:
#Veri okuma
def read_test2_data(folder_name):
    mails = []
    filenames=[]
    #sadece txt olan dosya isimlerini al
    for name in os.listdir(folder_name):
        if name.endswith(".txt"):
            print(name)
            filenames.append(name)

    #oku ve mails içine at
    for filename in filenames:
        with open(os.path.join(folder_name, filename), 'r', encoding='latin-1') as f:
            messageBody = f.read()
            mails.append(messageBody)

    return mails

In [35]:
spam_test_list, ham_test_list = read_data('data_spam_ham_emails\\development\\')

test(spam_test_list, ham_test_list)

TP: 72
FP: 0
TN: 90
FN: 18
Accuracy: %90
Precision: %100
Recall: %80
F1 Score: %80


In [44]:
spam_test,ham_test = read_test_data('data_spam_ham_emails\\unseen_data_1\\')
test(spam_test, ham_test)

TP: 7
FP: 1
TN: 10
FN: 2
Accuracy: %85
Precision: %87
Recall: %77
F1 Score: %77


In [65]:
spam_test2 = read_test2_data('data_spam_ham_emails\\unseen1_karisik_cevap_excelde\\')
predict(spam_test2)

01.txt
02.txt
03.txt
04.txt
05.txt
06.txt
07.txt
08.txt
09.txt
10.txt
11.txt
12.txt
Spam
Spam
Spam
Spam
Spam
Ham
Spam
Spam
Ham
Ham
Spam
Spam


In [66]:
spam_test3 = read_test2_data('data_spam_ham_emails\\unseen2_tamamen_ham_test_data\\')
predict(spam_test3)

01.txt
02.txt
03.txt
04.txt
05.txt
06.txt
07.txt
08.txt
09.txt
10.txt
11.txt
12.txt
Ham
Ham
Ham
Ham
Ham
Ham
Ham
Ham
Ham
Ham
Ham
Ham
